In [ ]:
!pip3 install urllib3==1.25.4
!pip install alignment
!pip3 install transformers==2.8.0

In [ ]:
def train_test_files_split(f='df_only_latest.txt'):
  file = open(f)
  lines = file.read().split('\n')
  test_size = int(len(lines) * 0.15)
  test = lines[:test_size]
  train = lines[test_size:]
  train_file = open('train.txt', 'w')
  test_file = open('test.txt', 'w')
  print('\n'.join(test), file=test_file)
  print('\n'.join(train), file=train_file)
  test_file.close()
  train_file.close()

In [ ]:
train_test_files_split('df.txt')

In [ ]:
#sberbank-ai/rugpt3medium_based_on_gpt2
!mkdir gpt
!python pretrain_transformers.py \
    --output_dir=/content/drive/MyDrive/gpt \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3large_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=test.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 5 \
    --learning_rate 5e-5 \
    --num_train_epochs 40 \
    --block_size 120 \
    --overwrite_output_dir \
    --line_by_line \
    --save_steps 1000000 \
    --check_acc_step 1000

In [ ]:
!mkdir gpt2
!python pretrain_transformers.py \
    --output_dir=gpt2 \
    --model_type=gpt2 \
    --model_name_or_path=gpt \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=test.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 5 \
    --learning_rate 5e-5 \
    --num_train_epochs 40 \
    --block_size 120 \
    --overwrite_output_dir \
    --line_by_line \
    --save_steps 1000000 \
    --check_acc_step 1000

429

In [ ]:
model = model.cuda() # Выполнить, если доступно gpu

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
tokens = []
simple = ['я', 'мы', 'нас', 'нами', 'мне', 'мой', 'мое', 'моё', 'моего', 'наш', 'нашего', 'нашему', 'мной', 'нами']
for word in simple:
  options = []
  options.append(tokenizer.encode(word)[0])
  options.append(tokenizer.encode(word.capitalize())[0])
  options.append(tokenizer.encode(' ' + word)[0])
  options.append(tokenizer.encode(' ' + word.capitalize())[0])
  tokens.extend(options)

In [ ]:
tokens = []
difficult = ['он', 'они', 'их', 'ими', 'ему', 'его', 'ей', 'ее', 'её', 'им', 'него', 'нее', 'неё']
for word in difficult:
  options = []
  options.append(tokenizer.encode(word)[0])
  options.append(tokenizer.encode(word.capitalize())[0])
  options.append(tokenizer.encode(' ' + word)[0])
  options.append(tokenizer.encode(' ' + word.capitalize())[0])
  tokens.extend(options)

In [ ]:
from alignment.sequence import Sequence
from alignment.vocabulary import Vocabulary
from alignment.sequencealigner import SimpleScoring, GlobalSequenceAligner

def align(a, b):
    a =  Sequence([-1, -1, -1, -1] + a + [-1, -1, -1, -1])
    b = Sequence([-1, -1, -1, -1] + b + [-1, -1, -1, -1])
    v = Vocabulary()
    aEncoded = v.encodeSequence(a)
    bEncoded = v.encodeSequence(b)
    scoring = SimpleScoring(2, -1)
    aligner = GlobalSequenceAligner(scoring, -2)
    score, encodeds = aligner.align(aEncoded, bEncoded, backtrace=True)
    p = v.decodeSequenceAlignment(encodeds[0])
    return p.first[4:-4], p.second[4:-4]

[0.5, 0.5, 5]

In [ ]:
MAX_LENGTH = int(10000)
def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length

In [ ]:
repetition_penalty = 1.0
device = 'cuda' # Поменять при необходимости
k = 50
p = 0.5
num_return_sequences = 1
stop_token = '</s>'

def predict_raw(model, tokenizer, prompt_text):
  length = adjust_length_to_model(100, max_sequence_length=model.config.max_position_embeddings)
  temperature = 1.0
  generated_sequences = []
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(device)
  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=length + len(encoded_prompt[0]),
      temperature=temperature,
      top_k=k,
      top_p=p,
      repetition_penalty=repetition_penalty,
      do_sample=True,
      num_return_sequences=num_return_sequences,
  )
  if len(output_sequences.shape) > 2:
      output_sequences.squeeze_()
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
      generated_sequence = generated_sequence.tolist()
      text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

      text = text[: text.find(stop_token) if stop_token else None]
      total_sequence = (
          prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
      )
      generated_sequences.append(total_sequence)
  return generated_sequences[0]


In [ ]:
def predict(text, model=model, tokenizer=tokenizer):
  return predict_raw(model, tokenizer, f'in: {text} [end] out: ').split('[end] out: ')[1]

In [ ]:
import torch
from alignment.sequence import Sequence
from alignment.vocabulary import Vocabulary
from alignment.sequencealigner import SimpleScoring, GlobalSequenceAligner

def align(a, b):
  a =  Sequence([-1, -1, -1, -1] + a + [-1, -1, -1, -1])
  b = Sequence([-1, -1, -1, -1] + b + [-1, -1, -1, -1])
  v = Vocabulary()
  aEncoded = v.encodeSequence(a)
  bEncoded = v.encodeSequence(b)

  scoring = SimpleScoring(2, -1)
  aligner = GlobalSequenceAligner(scoring, -2)
  score, encodeds = aligner.align(aEncoded, bEncoded, backtrace=True)

  p = v.decodeSequenceAlignment(encodeds[0])
  return p.first[4:-4], p.second[4:-4]

def split_by_words(sentence):
  words = []
  i = 0
  last_stop = 0

  while i < len(sentence):
    if sentence[i].isalpha():
      i += 1
      continue
    elif sentence[i].isspace():
      if last_stop < i:
        words.append(sentence[last_stop:i])
      i += 1
      last_stop = i
    else:
      if last_stop < i:
        words.append(sentence[last_stop:i])
      words.append(sentence[i])
      i += 1
      last_stop = i
  return words

def get_pair(a, b, pad_id = 0, device='cuda'):
  if pad_id is None:
    pad_id = 0
  a, b = align(a, b)
  for i in range(len(a)):
    if a[i] == '-' or a[i] is None:
      a[i] = pad_id
  for i in range(len(b)):
    if b[i] == '-'  or b[i] is None:
      b[i] = pad_id
  if device is None:
    return a, b
  return torch.tensor(a).view(1, -1).to(device), torch.tensor(b).view(1, -1).to(device)

In [ ]:
def read_data(path):
  result = []
  with open(path, 'r') as file:
    for line in file:
      inp, out = line[3:-5].split(' [end] out: ')
      inp = inp.strip()
      out = out.strip()
      if inp[-1] != '.':
        inp += '.'
      if out[-1] != '.':
        out += '.'
      result.append((inp, out))
  return result



In [ ]:
def calc_accuaracy(model, tokenizer, pairs):
  simple = 'Я я мы Мы мне Мне мой Мой Моего моего Нам нам'.split(' ')
  model.eval()
  with torch.no_grad():
    correct = 0 
    total = 0
    correct_changed = 0 
    total_changed = 0

    correct_difficult_changed = 0
    total_difficult_changed = 0

    for pair in pairs:
      if True:
        input, target = pair
        predicted = predict(input)
        input2 = input
        input, target = get_pair(split_by_words(input), split_by_words(target), '<pad>', device=None)
        input, predicted = get_pair(split_by_words(input2), split_by_words(predicted), '<pad>', device=None)

        while len(target) > len(input):
          target = target[:-1]

        while len(input) > len(target):
          input = input[:-1]
        
        while len(predicted) > len(target):
          predicted = predicted[:-1]
          total += 1

        while len(predicted) < len(target):
          target = target[:-1]
          input = input[:-1]
          total += 1

        predicted = np.array(predicted)
        input = np.array(input)
        target = np.array(target)

        correct_tensor = predicted == target
        is_changed_tensor = input != target

        correct += correct_tensor.sum()
        total += predicted.shape[0]

        correct_changed += (correct_tensor * is_changed_tensor).sum()
        total_changed += is_changed_tensor.sum()

        for token in simple:
          is_changed_tensor *= (input != token)

        
        correct_difficult_changed += (correct_tensor * is_changed_tensor).sum()
        total_difficult_changed += is_changed_tensor.sum()


  model.train()
  print(total, total_changed, total_difficult_changed)
  return correct / total, correct_changed / total_changed, correct_difficult_changed / total_difficult_changed

In [ ]:
import torch
pairs = read_data('train.txt')
acc = calc_accuaracy(model, tokenizer, pairs)
print(acc)

# Проверяем полученнную модель

In [ ]:
predict('Ваня мне помог, но я его проигнорировал')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


' Ваня ему помог, но он Ваню проигнорировал' '

In [ ]:
predict('Вчера мы думали о том чтобы купить пень но я понял что он слишком дорогой.')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


' Вчера они думали о том чтобы купить пень но она поняла что пень слишком дорогой. '

In [ ]:
predict('Я захотел поднять вазу. Она была очень хрупкой и стояла очень высоко. Пришел слон и сломал её т. к. у него было плохое настроение.')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


' Он захотел поднять вазу. Ваза была очень хрупкой и стояла очень высоко. Пришел слон и сломал вазу т. к. у слона было плохое настроение. '

In [ ]:
predict('Недавно я у Пети купил машину. Он мне сбросил цену на неё.')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


' Недавно он у Пети купил машину. Петя ему сбросил цену на машину. '

In [ ]:
predict('Я пришел к Ване. Он меня ударил')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


' Он пришел к Ване. Ваня меня ударил '

In [ ]:
predict('Я лечу в Индию')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


' Он летит в Индию '

In [ ]:
predict('Я строитель. Мама хорошая. Она меня любит и много делает для меня')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


' Он строитель. Мама хорошая. Мама его любит и много делает для него '